In [ ]:
include("table.jl")

In [ ]:
results = load_results()

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
df = results2df(results)

In [ ]:
sort(df[(df.model .== "EQ") .& (df.train_mat .== "maxCOV"),:], [:train_d, :train_gtype, :train_mat])

In [ ]:
sort(df[(df.model .== "EQ") .& (df.train_mat .== "COR"),:], [:train_d, :train_gtype, :train_mat])

In [ ]:
sort(df[(df.model .== "CNN") .& (df.train_mat .== "maxCOV"),:], [:train_d, :train_gtype, :train_mat])

# Test dataframe selection

And exploring the data

In [ ]:
df[1:4,["model","train_d"]]

In [ ]:
# sum(df.raw)

# I don't need train_k, because all of them are 1s
df = df[!, Not(All(:train_k, :train_noise))]

unique(df.train_mat)
unique(df.model)

In [ ]:
function select_sep_df(df, model)
    sort(df[df.model .== model,
                    # test_d is same as train_d
                    Not(All(:train_k, :train_noise, :test_d))],
                 [:train_mec, :train_gtype, :test_gtype, :train_d, :test_k])
end

function select_ensemble_df(df, model)
    subdf = sort(df[df.model .== model, Not(All(:train_k, :train_noise, r"train_"))],
                 [:test_gtype, :test_d, :test_k])
    show(subdf, allrows=true)
    nothing
end

In [ ]:
# 1. all the models in separate mode
select_sep_df(df, "FC")

In [ ]:
select_sep_df(df, "EQ")

# Pre-defined tables

In [ ]:
table_cmp_baseline(df)

In [ ]:
table_ER24(df)

In [ ]:
table_raw(df)

In [ ]:
table_transfer_k(df)

In [ ]:
table_transfer_noise(df)

In [ ]:
table_transfer_gtype(df)

In [ ]:
table_ensemble(df)

# Developing the tables

## Table 1: compare EQ/CNN/FC with the literature

- [ ] TODO literature in baseline.py
- [ ] TODO I want to probably generate the final layout to use in the paper.

I need:
- add sep lines
- break into "two columns"

Possibly through:
- I need to generate nice output here
- generate latex code to copy&paste into paper

In [ ]:
function table_cmp_literature(df)
    # comparison of EQ with FC and CNN (and random) baselines
    # only show d=10,15,20, SF and ER graphs, k=1
    sel0 = (df.train_mat .== "maxCOV")
    sel1 = in.(df.model, Ref(["FC", "EQ", "CNN"]))
    sel2 = in.(df.train_d, Ref([10, 20, 50, 100]))
    selector = (sel0 .& sel1 .& sel2
                .& (df.train_gtype .== df.test_gtype)
                .& in.(df.train_gtype, Ref(["SF", "ER"]))
                .& (df.test_noise .== "Gaussian")
                .& (df.train_mec .== "Linear")
                .& (df.test_mec .== "Linear")
#                 .& (df.train_mat .== "COR")
#                 .& (df.test_mat .== "COR")
                .& (df.test_k .== 1))

    sort(df[selector,
            All(:model, :train_d, :prec, :recall, :shd)],
         [:train_d, :model])
end

table_cmp_literature(df)

## Table 2: tranfer experiments

- [ ] different noise model
- [ ] different causal strength `k`
- [ ] different types of graph
- [ ] transfer to very large graphs (d=200,300,400), and compare to GES
- [ ] ensemble d=10,15,20, and test on d=10,15,20,30,50,80

In [ ]:
# FIXME no train_k recorded in the new experimetns.
# FIXME all test_k is 1
df[:train_k]

In [ ]:
function table_transfer_noise(df)
    # - train on Gaussian, test on Exp, Gumbel
    selector = ((df.model .== "EQ")
        .& (df.train_mat .== "maxCOV")
                .& in.(df.train_gtype, Ref(["SF", "ER"]))
                .& (df.train_gtype .== df.test_gtype)
                .& (df.test_k .== 1)
                .& (df.train_mec .== "Linear")
                .& (df.test_mec .== "Linear"))
    sort(df[selector,
            Not(All(
                    # CAUTION I need to show this to tell that the training and
                    # testing noise are different :train_noise,
                    :test_d, :test_k,
                    :train_mec, :test_mec,
                    :test_gtype,
                    :train_mat, :test_mat))],
         [:train_gtype, :model, :train_d, :test_noise])
end
table_transfer_noise(df)

In [ ]:
function table_transfer_gtype(df)
    # - train on ER, test on SF
    selector = ((df.model .== "EQ")
                # TODO ER2 ER4 SF2 SF4
                .& in.(df.train_gtype, Ref(["ER", "SF"]))
        # TODO this is not in the new experiments
                .& (df.train_gtype .!= df.test_gtype)
#                 .& (df.test_noise .== "Gaussian")
                .& (df.test_k .== 1)
                .& (df.train_mec .== "Linear")
                .& (df.test_mec .== "Linear")
                .& (df.train_mat .== "maxCOV"))
    sort(df[selector,
            Not(All(
                    :test_d, :test_k,
                    :train_mec, :test_mec, :test_noise,
                    :train_mat, :test_mat))],
         [:train_gtype, :model, :test_gtype, :train_d])
end
table_transfer_gtype(df)

In [ ]:

function table_transfer_k(df)
    # - train on k=1, test on k=1,2,4
    selector = ((df.model .== "deep-EQ")
                .& (df.test_noise .== "Gaussian")
                .& (df.train_gtype .== "SF")
                .& (df.test_gtype .== "SF")
                .& (df.train_mec .== "Linear")
                .& (df.test_mec .== "Linear")
                .& (df.train_mat .== "COR")
                .& (df.test_mat .== "COR"))
    sort(df[selector,
            Not(All(:train_k, :train_noise,
                    :test_d,
                    :train_mec, :test_mec, :test_noise,
                    :train_mat, :test_mat))],
         [:train_gtype, :model, :test_gtype, :train_d, :test_k])
end
table_transfer_k(df)

In [ ]:
function table_ensemble(df)
    selector = (in.(df.model, Ref(["deep-EQ-ER-ensemble",
                                   "deep-EQ-SF-ensemble"]))
                .& (df.test_k .== 1)
                .& (df.test_mec .== "Linear")
                .& (df.test_mat .== "COR"))
    sort(df[selector,
            Not(All(:train_k, :train_noise,
                    :test_k,
                    r"train_",
                    :train_mec, :test_mec, :test_noise,
                    :train_mat, :test_mat))],
         [:model, :test_gtype, :test_d])
end
table_ensemble(df)

# Testing displaysize

In [ ]:
displaysize()

In [ ]:
# show all of df
show(df[1:3,:], allcols=true)

In [ ]:
IOContext(stdout, :displaysize=>(30,100)) |> displaysize